# Notebook pour nettoyer les données BigQuery

* __Description__: Notebook pour nettoyer les données des tables post et comment dans BigQuery
* __Source__: Tables comment et post (BigQuery) 
* __Output__: Tables comment et post (BigQuery) 
* __Auteur__: Corentin TIMAL et Camille MATTHIEU
* __Date de création__: 15/09/2022
* __Date de mise à jour__: 15/09/2022

## Import des outils

In [ ]:
%run /notebook/Libs/requete_lib.ipynb

## Nettoyage

Cette fonction permet de supprimer les doublons dans les tables post et comment suite au mise à jour des données

In [1]:
def nettoyage_complet():
    """
    Permet d'exécuter différentes requêtes SQL dans BigQuery pour supprimer les doublons présents dans les tables post et comment
    """
    
    ## POST
    
    # Eliminer les doublons de post, en ne gardant que celui avec extraction_utc le plus élevé
    sql = """CREATE OR REPLACE TABLE `mimetic-coral-355913.dwh.post` AS
    (
    WITH post_unique AS (
      SELECT * FROM `mimetic-coral-355913.dwh.post` AS post
      LEFT OUTER JOIN (
        SELECT t.id_post AS id_post2, MAX(t.extraction_utc) AS extraction_latest FROM `mimetic-coral-355913.dwh.post` AS t
        GROUP BY t.id_post
      ) ON post.id_post = id_post2
    )
    SELECT id_post, id_subreddit, subreddit, id_author, author, num_comments, subreddit_subscribers, upvote_ratio, ups, downs, score, created_utc, extraction_utc, kind, score_jigsaw FROM post_unique
    WHERE extraction_utc = extraction_latest
    )
    ;"""
    requete_sql_bigquery(sql)
    
    ## COMMENT
    
    # Eliminer les lignes avec type_content = null
    sql = """DELETE FROM `mimetic-coral-355913.dwh.comment`
    WHERE type_content IS NULL;"""
    requete_sql_bigquery(sql)

    # Eliminer les commentaires dont le contenu est "[effacé]"
    sql = f"""DELETE FROM `mimetic-coral-355913.dwh.comment`
    WHERE content IS NULL;"""
    requete_sql_bigquery(sql)
    
#     # Eliminer les commentaires dont le contenu est "[supprimé]"
#     sql = f"""DELETE FROM `mimetic-coral-355913.dwh.comment`
#     WHERE content = 'deleted';"""
#     requete_sql_bigquery(sql)

    # Eliminer les doublons en ne gardant que les plus récents (soit l'extraction_utc le plus élevé)
    sql = """CREATE OR REPLACE TABLE `mimetic-coral-355913.dwh.comment` AS
    (
    WITH comment_unique AS (
      SELECT * FROM `mimetic-coral-355913.dwh.comment` AS comment
      LEFT OUTER JOIN (
        SELECT t.id_comment AS id_comment2, t.type_content AS type_content2, MAX(t.extraction_utc) AS extraction_latest FROM `mimetic-coral-355913.dwh.comment` AS t
        GROUP BY t.id_comment, t.type_content
      ) ON (comment.id_comment = id_comment2 AND comment.type_content = type_content2)
    )
    SELECT id_comment, id_post, id_author, author, content, type_content, ups, downs, score, created_utc, extraction_utc, score_jigsaw FROM comment_unique
    WHERE extraction_utc = extraction_latest
    )
    ;"""
    requete_sql_bigquery(sql)

    # Eliminer les doublons restants (i.e. les commentaires avec les mêmes dates d'extraction)
    sql = """CREATE OR REPLACE TABLE `mimetic-coral-355913.dwh.comment` AS
    (
    SELECT DISTINCT * FROM `mimetic-coral-355913.dwh.comment`
    )
    ;

    CREATE OR REPLACE TABLE `mimetic-coral-355913.dwh.comment` AS
    (
    WITH comment_unique_2 AS (
      SELECT * FROM `mimetic-coral-355913.dwh.comment` AS comment
      LEFT OUTER JOIN (
        SELECT t.id_comment AS id_comment2, t.type_content AS type_content2, MAX(t.score) AS score_max FROM `mimetic-coral-355913.dwh.comment` AS t
        GROUP BY t.id_comment, t.type_content
      ) ON (comment.id_comment = id_comment2 AND comment.type_content = type_content2)
    )
    SELECT id_comment, id_post, id_author, author, content, type_content, ups, downs, score, created_utc, extraction_utc, score_jigsaw FROM comment_unique_2
    WHERE score = score_max
    )
    ;"""
    requete_sql_bigquery(sql)
